## Demonstrate how to run sparql queries against the OpenRiskNet registry

This notebook demonstrates how to run a SparQL query against the OpenRiskNet registry. The registry at the moment only returns json formatted output and thus this example uses the requests library and puts the results directly into a dict of service names to pandas dataframes for further anlysis instead of using a specialized sparql library.

In [1]:
!pip install pandas

In [2]:
import requests
import pprint
import pandas

In [3]:
query = """
# Select all endoints that use a query parameter that was assigned the CHEMINF_000018 id
# (smiles). Not that for parameters the mapping of json keys to ontology terms in 
# the x-onr-@context section does not do anything because the name of the parameter is 
# not specified as a json key but is just a string value. The author of a service has to
# explicitly annotate a parameter with x-orn-@id tag of the ontology term in question for
# this to work.

PREFIX orn: <http://openrisknet.org/schema#>
SELECT ?endpointnode ?relativeendpointpath ?endpointverbnode 
{ 
# start query at the "paths" predicate
?n1 orn:paths ?n2 . 

# one level down are the endpoints - we select the predicate which is the relative endpoint path (e.g. "/predict")
# here but this looks a bit weird as it is an IRI with the usual http://openrisknet.org/schema# prefix
# (e.g. "http://openrisknet.org/schema#/predict")
# If service authors opt to use x-orn-@id on their endpoints (or endpoint verbs), they can map the IRI
# of the node to the correct URL of the service endpoint so it can be queried there directly - but this
# only works if the id has been assigned correctly by the service author, otherwise you get an anonymous node id
?n2 ?relativeendpointpath ?endpointnode. 
?endpointnode ?verb ?endpointverbnode.

# Go one level down into the parameters section and match if the id of the parameter
# is the ontology term that we are looking for. Alternatively you could match based on 
# text or a regex here so you can find services where the parameter has not been tagged
# with this ontology term but then you rely on text based matching.
?endpointverbnode orn:parameters <http://semanticscience.org/resource/CHEMINF_000018>
}
"""

In [4]:
response = requests.get("https://registry.prod.openrisknet.org/api/sparql", params={"query": query})

In [5]:
response.raise_for_status()

In [6]:
json = response.json()
#pprint.pprint(json)

In [7]:
def mapServiceResult(serviceDict):
    result = pandas.DataFrame(data=serviceDict['Result'][1]['ResultValues'], columns=serviceDict['Result'][1]['Variables'])
    return serviceDict['ServiceName'], result

In [8]:
dictOfDataframes = dict(map(mapServiceResult, json))

In [9]:
for key, value in dictOfDataframes.items():
    display(key)
    display(value)

'BridgeDb identifier mapping service'

,endpointnode,relativeendpointpath,endpointverbnode,n1,n2,verb


'Lazar REST Service'

,endpointnode,relativeendpointpath,endpointverbnode,n1,n2,verb


'EPA-ToxCastV3.1-ACEA_ER_80hr_summary_tcpl'

...

,endpointnode,relativeendpointpath,endpointverbnode,n1,n2,verb


'JGU WEKA REST Service'

,endpointnode,relativeendpointpath,endpointverbnode,n1,n2,verb


'ChemIdConvert - Chemical identifier conversion service'

,endpointnode,relativeendpointpath,endpointverbnode,n1,n2,verb
0,_:b70,http://openrisknet.org/schema#/asSvg,http://chemidconvert.prod.openrisknet.org/v1/a...,,,
1,_:b373,http://openrisknet.org/schema#/molWeight,http://chemidconvert.prod.openrisknet.org/v1/m...,,,
2,_:b487,http://openrisknet.org/schema#/smiles/to/cas,http://chemidconvert.prod.openrisknet.org/v1/s...,,,
3,_:b512,http://openrisknet.org/schema#/smiles/to/inchi,http://chemidconvert.prod.openrisknet.org/v1/s...,,,
4,_:b529,http://openrisknet.org/schema#/smiles/to/inchikey,http://chemidconvert.prod.openrisknet.org/v1/s...,,,
5,_:b546,http://openrisknet.org/schema#/smiles/to/names,http://chemidconvert.prod.openrisknet.org/v1/s...,,,
